<a href="https://colab.research.google.com/github/CaduPereira0921/chatbot-floricultura-bella/blob/main/Chatbot_Floricultura_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baixando as bibliotecas necessárias e testando a API_KEY

In [1]:
!pip install -q openai

In [3]:
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")

if not api_key:
    raise RuntimeError("OPENAI_API_KEY não encontrada no Secrets do Colab")

print("API Key carregada com sucesso")


API Key carregada com sucesso


In [4]:
from openai import OpenAI

client = OpenAI(api_key=api_key)


In [5]:
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": "Em uma frase, o que é um chatbot?"
        }
    ],
    max_tokens=30
)

print("Resposta do modelo:")
print(response.choices[0].message.content)


Resposta do modelo:
Um chatbot é um programa de computador que simula uma conversa com usuários por meio de mensagens de texto ou voz.


# Inserindo contexto ao Chatbot e exemplos de resposta

In [6]:
import json

# carregar dados da floricultura
with open("config_lojaflores.json", "r", encoding="utf-8") as f:
    dados_loja = json.load(f)

# carregar template de contexto
with open("context_template.txt", "r", encoding="utf-8") as f:
    template_contexto = f.read()

# preencher o template com os dados do JSON
contexto_floricultura = template_contexto.format(**dados_loja)

print("Contexto carregado com sucesso")


Contexto carregado com sucesso


In [7]:
def chatbot_floricultura(pergunta):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": contexto_floricultura
            },
            {
                "role": "user",
                "content": pergunta
            }
        ],
        max_tokens=120
    )

    return response.choices[0].message.content


In [8]:
print(chatbot_floricultura("Qual o horário de funcionamento?"))


Nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h.


In [9]:
print(chatbot_floricultura("Quais as flores disponíveis?"))


Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se precisar de algo mais, estou à disposição!


In [10]:
print(chatbot_floricultura("Quanto custa um buquê de rosas vermelhas?"))


Desculpe, não tenho informações sobre preços específicos no momento. Por favor, entre em contato conosco diretamente pelo telefone (11) 98765-4321 para obter mais informações sobre preços e disponibilidade.


# Alterando a temperatura do chatbot

In [16]:
def chatbot_floricultura(pergunta, temperatura=1.0):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": contexto_floricultura
            },
            {
                "role": "user",
                "content": pergunta
            }
        ],
        temperature=temperatura,
        max_tokens=120
    )

    return response.choices[0].message.content


In [17]:
print("\nTemperatura 1.0")
print(chatbot_floricultura("Quais flores vocês vendem?", temperatura=1.0))




Temperatura 1.0
Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se quiser saber mais ou ver nosso catálogo completo, visite nosso site www.floresbella.com.br!


# Adicionando memória ao chatbot

In [18]:
# histórico inicial da conversa
historico_conversa = [
    {
        "role": "system",
        "content": contexto_floricultura
    }
]


In [19]:
def chatbot_floricultura(pergunta, temperatura=0.2):
    global historico_conversa

    # adiciona a pergunta do usuário ao histórico
    historico_conversa.append({
        "role": "user",
        "content": pergunta
    })

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico_conversa,
        temperature=temperatura,
        max_tokens=120
    )

    resposta = response.choices[0].message.content

    # adiciona a resposta do modelo ao histórico
    historico_conversa.append({
        "role": "assistant",
        "content": resposta
    })

    return resposta


In [20]:
print(chatbot_floricultura("Quais flores vocês vendem?"))


Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos.


In [21]:
print(chatbot_floricultura("E vocês fazem entrega?"))


Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00.


In [22]:
def resetar_conversa():
    global historico_conversa
    historico_conversa = [
        {
            "role": "system",
            "content": contexto_floricultura
        }
    ]
    print("Conversa reiniciada.")


# Validação de escopo

Por que essa é a melhor próxima função?

Porque ela mostra, de forma simples, que o chatbot não sabe tudo e que o comportamento pode ser controlado pelo desenvolvedor.

**IA não é magia, é engenharia de decisões**

E isso evita um dos maiores erros conceituais dos iniciantes:

*“O modelo responde qualquer coisa porque ele quer.”*

In [23]:
topicos_permitidos = [
    "flores",
    "arranjos",
    "entrega",
    "horário",
    "pagamento",
    "endereço",
    "telefone",
    "promoção"
]


In [24]:
def pergunta_dentro_do_escopo(pergunta):
    pergunta = pergunta.lower()
    return any(topico in pergunta for topico in topicos_permitidos)


In [25]:
def chatbot_floricultura(pergunta, temperatura=0.2):
    global historico_conversa

    if not pergunta_dentro_do_escopo(pergunta):
        return (
            "Desculpe, não tenho informações sobre isso no momento. "
            "Por favor, entre em contato conosco diretamente para obter mais ajuda."
        )

    historico_conversa.append({"role": "user", "content": pergunta})

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico_conversa,
        temperature=temperatura,
        max_tokens=120
    )

    resposta = response.choices[0].message.content
    historico_conversa.append({"role": "assistant", "content": resposta})

    return resposta


In [26]:
print(chatbot_floricultura("Quais flores vocês vendem?"))


Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos.


In [27]:
print(chatbot_floricultura("Qual o time ganhou o jogo ontem?"))


Desculpe, não tenho informações sobre isso no momento. Por favor, entre em contato conosco diretamente para obter mais ajuda.


# Limite de histórico

Até agora, o chatbot: guarda tudo o que foi dito e reenvia tudo ao modelo a cada pergunta.

**Problema real:**

- o prompt cresce

- o custo aumenta

- o modelo pode “se confundir”

**Solução: limitar o histórico.**

O chatbot não precisa lembrar de tudo, só do que é relevante recentemente.

In [28]:
MAX_INTERACOES = 4
# 4 interações = 4 perguntas + 4 respostas


In [29]:
#função para aplicar o corte do histórico

def limitar_historico(historico, max_interacoes):
    """
    Mantém:
    - a mensagem system (sempre)
    - as últimas N interações (user + assistant)
    """
    system_msg = historico[0]
    restante = historico[1:]

    # cada interação tem 2 mensagens (user + assistant)
    max_msgs = max_interacoes * 2

    if len(restante) > max_msgs:
        restante = restante[-max_msgs:]

    return [system_msg] + restante


In [34]:
#chatbot com histórico limitado

def chatbot_floricultura(pergunta, temperatura=0.2):
    global historico_conversa

    if not pergunta_dentro_do_escopo(pergunta):
        return (
            "Desculpe, não tenho informações sobre isso no momento. "
            "Por favor, entre em contato conosco diretamente para obter mais ajuda."
        )

    # adiciona pergunta
    historico_conversa.append({
        "role": "user",
        "content": pergunta
    })

    # limita antes da chamada
    historico_conversa = limitar_historico(
        historico_conversa,
        MAX_INTERACOES
    )

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico_conversa,
        temperature=temperatura,
        max_tokens=120
    )

    resposta = response.choices[0].message.content

    # adiciona resposta
    historico_conversa.append({
        "role": "assistant",
        "content": resposta
    })

    # 🔴 limita novamente após a resposta
    historico_conversa = limitar_historico(
        historico_conversa,
        MAX_INTERACOES
    )

    return resposta



In [44]:
print(chatbot_floricultura("Quais flores vocês vendem?"))
print(chatbot_floricultura("Vocês fazem entrega?"))
print(chatbot_floricultura("Quanto custa a entrega?"))
print(chatbot_floricultura("Quais formas de pagamento vocês aceitam?"))
print(chatbot_floricultura("Qual é o endereço da loja?"))
print(chatbot_floricultura("Quem ganhou o jogo ontem?"))
print(chatbot_floricultura("Qual é a capital da França?"))

Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas e Cravos.
Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00.
A taxa de entrega é de R$15,00 para toda a região de São Paulo (Capital).
Aceitamos as seguintes formas de pagamento: Cartão de Crédito, Pix e Dinheiro.
Nossa loja está localizada na Avenida das Flores, 456 - Centro.
Desculpe, não tenho informações sobre isso no momento. Por favor, entre em contato conosco diretamente para obter mais ajuda.
Desculpe, não tenho informações sobre isso no momento. Por favor, entre em contato conosco diretamente para obter mais ajuda.


In [45]:
len(historico_conversa)


9

In [46]:
for msg in historico_conversa:
    print(msg["role"], "->", msg["content"][:400])


system -> Você é um assistente virtual prestativo da floricultura Flores Bella.

Nossa loja está localizada em Avenida das Flores, 456 - Centro e nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h. Você pode nos contatar pelo telefone (11) 98765-4321 ou pelo email contato@floresbella.com.br. Visite nosso site em www.floresbella.com.br para ver nosso catálogo completo e o
user -> Vocês fazem entrega?
assistant -> Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00.
user -> Quanto custa a entrega?
assistant -> A taxa de entrega é de R$15,00 para toda a região de São Paulo (Capital).
user -> Quais formas de pagamento vocês aceitam?
assistant -> Aceitamos as seguintes formas de pagamento: Cartão de Crédito, Pix e Dinheiro.
user -> Qual é o endereço da loja?
assistant -> Nossa loja está localizada na Avenida das Flores, 456 - Centro.


# Conectar a um Banco de Dados: Persistência

**Até agora:**

- o histórico vive na memória

- ao reiniciar o notebook → tudo se perde

**Com persistência:**

- cada conversa é salva

- o chatbot pode continuar uma conversa

- lembrar interações passadas

- simular múltiplos usuários

# Usuário → Chatbot → SQLite → Chatbot → Resposta


In [47]:
#criar (ou abrir) o banco SQLite

import sqlite3

conn = sqlite3.connect("chatbot_floricultura.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS mensagens (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    role TEXT,
    content TEXT
)
""")

conn.commit()


In [48]:
#funções de persistência

def salvar_mensagem(session_id, role, content):
    cursor.execute(
        "INSERT INTO mensagens (session_id, role, content) VALUES (?, ?, ?)",
        (session_id, role, content)
    )
    conn.commit()


In [49]:
#Carregar histórico da sessão

def carregar_historico(session_id):
    cursor.execute(
        "SELECT role, content FROM mensagens WHERE session_id = ? ORDER BY id",
        (session_id,)
    )
    rows = cursor.fetchall()

    return [{"role": r, "content": c} for r, c in rows]


Banco → lista → histórico do chatbot.

In [50]:
#iniciar uma nova sessão

import uuid

def nova_sessao():
    return str(uuid.uuid4())

session_id = nova_sessao()


In [52]:
#chatbot com histórico persistido + limite

def chatbot_floricultura(pergunta, session_id, temperatura=0.2):
    # carrega histórico do banco
    historico = carregar_historico(session_id)

    # se for primeira mensagem, injeta o system
    if not historico:
        historico.append({
            "role": "system",
            "content": contexto_floricultura
        })
        salvar_mensagem(session_id, "system", contexto_floricultura)

    # salva pergunta
    historico.append({"role": "user", "content": pergunta})
    salvar_mensagem(session_id, "user", pergunta)

    # aplica limite de histórico (como antes)
    historico = limitar_historico(historico, MAX_INTERACOES)

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico,
        temperature=temperatura,
        max_tokens=120
    )

    resposta = response.choices[0].message.content

    # salva resposta
    salvar_mensagem(session_id, "assistant", resposta)

    return resposta


In [53]:
print(chatbot_floricultura("Quais flores vocês vendem?", session_id))
print(chatbot_floricultura("Vocês fazem entrega?", session_id))
print(chatbot_floricultura("Quanto custa a entrega?", session_id))


Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se quiser, posso ajudar a escolher a melhor opção para você!
Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00. Se desejar, posso ajudar a fazer seu pedido!
A taxa de entrega é de R$15,00 para toda a região de São Paulo (Capital). Se precisar de mais alguma informação, estou à disposição!


In [54]:
print("Session ID:", session_id)
print(chatbot_floricultura("Quais flores vocês vendem?", session_id))


Session ID: 401476f0-8e33-4c08-867e-f06163c5d521
Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se quiser, posso ajudar a escolher a melhor opção para você!


In [55]:
sessao_1 = nova_sessao()
sessao_2 = nova_sessao()

print("Sessão 1:", sessao_1)
print(chatbot_floricultura("Quais flores vocês vendem?", sessao_1))

print("\nSessão 2:", sessao_2)
print(chatbot_floricultura("Vocês fazem entrega?", sessao_2))


Sessão 1: 03343dbf-4aac-44d6-92e2-410b00fcf6bf
Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se quiser, posso ajudar a escolher a melhor opção para você!

Sessão 2: 1d3e6dbb-7a95-4761-882f-f70e7b2d6399
Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00.


In [56]:
print(chatbot_floricultura("Quanto custa a entrega?", sessao_1))


A taxa de entrega é de R$15,00 para toda a região de São Paulo (Capital). Se precisar de mais alguma informação, estou à disposição!


In [58]:
!ls


chatbot_floricultura.db  context_template.txt
config_lojaflores.json	 sample_data


In [60]:
def mostrar_historico_banco(session_id):
    cursor.execute(
        "SELECT role, content FROM mensagens WHERE session_id = ? ORDER BY id",
        (session_id,)
    )
    rows = cursor.fetchall()

    if not rows:
        print("Nenhuma mensagem encontrada para esta sessão.")
        return

    print(f"Histórico da sessão {session_id}:\n")
    for role, content in rows:
        print(f"{role.upper()}: {content}")


In [61]:
print("Session ID:", session_id)
mostrar_historico_banco(session_id)


Session ID: 401476f0-8e33-4c08-867e-f06163c5d521
Histórico da sessão 401476f0-8e33-4c08-867e-f06163c5d521:

SYSTEM: Você é um assistente virtual prestativo da floricultura Flores Bella.

Nossa loja está localizada em Avenida das Flores, 456 - Centro e nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h. Você pode nos contatar pelo telefone (11) 98765-4321 ou pelo email contato@floresbella.com.br. Visite nosso site em www.floresbella.com.br para ver nosso catálogo completo e ofertas especiais.

Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Oferecemos também diversos tipos de arranjos florais, como: Buquês, Coroas de Flores, Arranjos de Mesa, Cestas de Café da Manhã, Decoração de Ambientes.

Entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00. Aceitamos as seguintes formas de pagamento: ['Cartão de Crédito', 'Pix', 'Dinheiro'].

Atualmente, 

In [62]:
print("Session ID:", session_id)
mostrar_historico_banco(session_id)


Session ID: 401476f0-8e33-4c08-867e-f06163c5d521
Histórico da sessão 401476f0-8e33-4c08-867e-f06163c5d521:

SYSTEM: Você é um assistente virtual prestativo da floricultura Flores Bella.

Nossa loja está localizada em Avenida das Flores, 456 - Centro e nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h. Você pode nos contatar pelo telefone (11) 98765-4321 ou pelo email contato@floresbella.com.br. Visite nosso site em www.floresbella.com.br para ver nosso catálogo completo e ofertas especiais.

Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Oferecemos também diversos tipos de arranjos florais, como: Buquês, Coroas de Flores, Arranjos de Mesa, Cestas de Café da Manhã, Decoração de Ambientes.

Entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00. Aceitamos as seguintes formas de pagamento: ['Cartão de Crédito', 'Pix', 'Dinheiro'].

Atualmente, 

*“Session ID não identifica o usuário. Session ID identifica a conversa.”*

In [64]:
session_1 = nova_sessao()
session_2 = nova_sessao()

print("Session 1:", session_1)
print(chatbot_floricultura("Quais flores vocês vendem?", session_1))

print("\nSession 2:", sessao_2)
print(chatbot_floricultura("Vocês fazem entrega?", session_2))

print("\nSession 1 novamente:")
print(chatbot_floricultura("Quanto custa a entrega?", session_1))


Session 1: 7c5ba100-7586-4bc1-9195-e5bd43a89f0b
Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Se quiser, posso ajudar a escolher a melhor opção para você!

Session 2: 47ba9b0b-7284-4a0f-a599-d9fa3512ad64
Sim, entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00.

Session 1 novamente:
A taxa de entrega é de R$15,00 para toda a região de São Paulo (Capital). Se precisar de mais alguma informação, estou à disposição!


# Listar todas as Sessions IDs do Banco de Dados

No banco, cada conversa é identificada por um session_id.
Visualizar todas as sessões significa responder:

- Quantas conversas existem?

- Quais são diferentes entre si?

- O que cada conversa contém?

In [65]:
#Ver todas as Sessions do nosso banco:
def listar_sessions():
    cursor.execute("""
        SELECT DISTINCT session_id
        FROM mensagens
        ORDER BY session_id
    """)
    sessions = cursor.fetchall()

    print("Sessions encontradas no banco:\n")
    for i, (sid,) in enumerate(sessions, start=1):
        print(f"{i}. {sid}")

    print(f"\nTotal de sessões: {len(sessions)}")


In [66]:
listar_sessions()


Sessions encontradas no banco:

1. 03343dbf-4aac-44d6-92e2-410b00fcf6bf
2. 1d3e6dbb-7a95-4761-882f-f70e7b2d6399
3. 401476f0-8e33-4c08-867e-f06163c5d521
4. 47ba9b0b-7284-4a0f-a599-d9fa3512ad64
5. 7c5ba100-7586-4bc1-9195-e5bd43a89f0b
6. 7d2b2efe-b027-4259-8136-da274e5d8f7a
7. f4770731-5096-4c43-8888-263a3383ea4b

Total de sessões: 7


In [71]:
#ver conteúdo específico de uma Session
def visualizar_sessao(session_id):
    cursor.execute("""
        SELECT role, content
        FROM mensagens
        WHERE session_id = ?
        ORDER BY id
    """, (session_id,))

    rows = cursor.fetchall()

    print(f"\nHistórico da sessão {session_id}:\n")
    for role, content in rows:
        print(f"{role.upper()}: {content}")


In [72]:
visualizar_sessao("47ba9b0b-7284-4a0f-a599-d9fa3512ad64")



Histórico da sessão 47ba9b0b-7284-4a0f-a599-d9fa3512ad64:

SYSTEM: Você é um assistente virtual prestativo da floricultura Flores Bella.

Nossa loja está localizada em Avenida das Flores, 456 - Centro e nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h. Você pode nos contatar pelo telefone (11) 98765-4321 ou pelo email contato@floresbella.com.br. Visite nosso site em www.floresbella.com.br para ver nosso catálogo completo e ofertas especiais.

Trabalhamos com uma grande variedade de flores, incluindo: Rosas, Lírios, Orquídeas, Girassóis, Tulipas, Margaridas, Cravos. Oferecemos também diversos tipos de arranjos florais, como: Buquês, Coroas de Flores, Arranjos de Mesa, Cestas de Café da Manhã, Decoração de Ambientes.

Entregamos em toda a região de São Paulo (Capital) com uma taxa de entrega de R$15,00. Aceitamos as seguintes formas de pagamento: ['Cartão de Crédito', 'Pix', 'Dinheiro'].

Atualmente, temos as seguintes promoções: Na compra de um bu

In [73]:
#vermos um resumo das sessions
def resumo_sessions():
    cursor.execute("""
        SELECT session_id, COUNT(*) as total_msgs
        FROM mensagens
        GROUP BY session_id
        ORDER BY total_msgs DESC
    """)

    rows = cursor.fetchall()

    print("Resumo das sessões:\n")
    for sid, total in rows:
        print(f"Session ID: {sid} | Mensagens: {total}")


In [74]:
resumo_sessions()


Resumo das sessões:

Session ID: 401476f0-8e33-4c08-867e-f06163c5d521 | Mensagens: 9
Session ID: f4770731-5096-4c43-8888-263a3383ea4b | Mensagens: 5
Session ID: 7c5ba100-7586-4bc1-9195-e5bd43a89f0b | Mensagens: 5
Session ID: 03343dbf-4aac-44d6-92e2-410b00fcf6bf | Mensagens: 5
Session ID: 7d2b2efe-b027-4259-8136-da274e5d8f7a | Mensagens: 3
Session ID: 47ba9b0b-7284-4a0f-a599-d9fa3512ad64 | Mensagens: 3
Session ID: 1d3e6dbb-7a95-4761-882f-f70e7b2d6399 | Mensagens: 3


# Iserindo um RAG no nosso chatbot

**RAG (Retrieval-Augmented Generation)** = buscar informação externa → injetar no prompt → gerar resposta

Vamosar utilizar a "política de entrega" como exemplo, pois ela não precisa sempre subir no prompt, apenas quando for necessária.


In [75]:
#Retrieval simples (sem embeddings)
def recuperar_politica_entrega(pergunta):
    if "entrega" in pergunta.lower():
        with open("politica_entrega.txt", "r", encoding="utf-8") as f:
            return f.read()
    return None


In [76]:
#Aumentar o prompt SOMENTE quando necessário
def chatbot_floricultura(pergunta, session_id, temperatura=0.2):
    historico = carregar_historico(session_id)

    if not historico:
        historico.append({"role": "system", "content": contexto_floricultura})

    # 🔴 RAG entra aqui
    contexto_extra = recuperar_politica_entrega(pergunta)

    if contexto_extra:
        historico.append({
            "role": "system",
            "content": f"Informação adicional:\n{contexto_extra}"
        })

    historico.append({"role": "user", "content": pergunta})

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=historico,
        temperature=temperatura,
        max_tokens=120
    )

    resposta = response.choices[0].message.content
    salvar_mensagem(session_id, "user", pergunta)
    salvar_mensagem(session_id, "assistant", resposta)

    return resposta


In [77]:
#pergunta sem RAG
print(chatbot_floricultura("Qual o horário de funcionamento?", session_id))


Nosso horário de funcionamento é Segunda a Sexta, das 8h às 19h; Sábados, das 8h às 14h. Estamos à disposição para atendê-lo nesse período!


In [78]:
#pergunta com RAG
print(chatbot_floricultura("Vocês fazem entrega aos domingos?", session_id))


Não, não realizamos entregas aos domingos e feriados. As entregas são feitas de Segunda a Sábado, dentro do nosso horário de funcionamento. Se precisar de mais alguma informação, estou à disposição!


**“RAG não é o modelo buscar informação. RAG é o sistema decidir o que o modelo precisa ler.”**